In [2]:
from FoldModellingPlugin.FoldModellingPlugin.optimisers import FourierSeriesOptimiser as FSO

/mnt/c/Users/rcha0044/LoopPhD/LOOP_ENV/env_18/lib/python3.8/site-packages/FoldModellingPlugin/FoldModellingPlugin/helper/_helper.py:105: SyntaxWarning: "is" with a literal. Did you mean "=="?
  x) if fold_frame is 1 else geological_feature.fold.fold_limb_rotation(x)
/mnt/c/Users/rcha0044/LoopPhD/LOOP_ENV/env_18/lib/python3.8/site-packages/FoldModellingPlugin/FoldModellingPlugin/helper/utils.py:216: SyntaxWarning: "is" with a literal. Did you mean "=="?
  x) if fold_frame is 1 else geological_feature.fold.fold_limb_rotation(x)


ImportError: cannot import name 'vonmises_fisher' from 'scipy.stats' (/mnt/c/Users/rcha0044/LoopPhD/LOOP_ENV/env_18/lib/python3.8/site-packages/scipy/stats/__init__.py)